# Welcome to your Database
## We will be Connecting Python code to a SQL Database
In this assignment we will be running a webform and will be adapting the code to use a database backend. Once we have sucessfully used a database backend we will then find and fix the most common bugs affecting webapp database and fixing the damage that they can cause to your database. Lets get started!

## What is a DB-API?

![](img/dbapi1.PNG)

In the first two notebooks we have gone though how to get a database to give up and output given a query and had used a black box to hide the bits about how a query gets the information. If you were following the course on Udacity the querys you enter were send to the web servers but now we will go into more depth about how the magic happens.

![](img/dbapi2.PNG)

Behind the Udacity web servers is python code that allows us to connect to an sqlite database using the DB-API method calls. These DB_API isnt a library but a standard for libraries to left python code connect to databases.  If you learnt he BD-API functions you can apply those to any database system. Below we have Database Systems and their corresponding libraries.

![](img/dbapi3.PNG)

In the VM we will be using the PostgreSQL.

![](img/dbapi4.PNG)

Above is an example of using python code with the DB-API and in this case sqlite3. The general rules can be applied to any database with the DB-API. 

## Query Example
1. We want to do is import the DB-API in the example above we use sqlite
2. Connect to your Database may have to state host name, username, password and other information. The connection will stay open until we close it.
3. Initialize  our cursor, the cursor is what actually runs query and fetches results. It is called cursor because when the data base give you results we want to scan though them like we would with a text cursor.
4. Execute a query using the cursor
5. Fetch all the results
6. (optional) if you insert data into our database we may want to commit() it here or is something went wrong rollback
6. Close our connection once we finish!

### Code to Create Table

In [3]:
import sqlite3
conn = sqlite3.connect('people.db')
cursor = conn.cursor()
cursor.execute("CREATE TABLE friends (name text, age integer)")
conn.commit()
conn.close()

### Code to Insert into a Table

In [4]:
import sqlite3
conn = sqlite3.connect('people.db')
cursor = conn.cursor()
cursor.execute("INSERT INTO friends VALUES ('Steve', 26)")
conn.commit()
conn.close()

In [ ]:
### Code to Query our Table

In [5]:
import sqlite3
conn = sqlite3.connect('people.db')
cursor = conn.cursor()
cursor.execute("SELECT * from friends ")
result = cursor.fetchall()
conn.close()

In [6]:
# print our results
print(result)

[('STEVE', 26)]


Lets fill up our sample database

In [10]:
conn = sqlite3.connect('people.db')
cursor = conn.cursor()
cursor.execute("""INSERT INTO friends (name, age) 
    VALUES ('Kim', 24),('Brian', 25), ('Stephanie', 22), ('ED', 26)""")
conn.commit()
conn.close()

Let read our data and sort by name. 

In [13]:
conn = sqlite3.connect('people.db')
cursor = conn.cursor()
cursor.execute("SELECT * from friends ORDER BY NAME")
rows = cursor.fetchall()
conn.close()

for row in rows:
    print("  ", row)

   ('Brian', 25)
   ('ED', 26)
   ('Kim', 24)
   ('STEVE', 26)
   ('Stephanie', 22)


For the second half of the notebook we wil be working in a vagrant file. Please make sure to have all the required instaill programs. Please start up your virtual machines in the relational-db directory.

## SQL Injection Attack
When working with our webforum we the first task we had was to modify add_post to use a progresql call and insert our message into the database.
Below is an example of what the code may look like

In [ ]:
import psycopg2

DBNAME = "forum"

def get_posts():
    """Return all posts from the 'database', most recent first."""
    db = psycopg2.connect(database=DBNAME)
    c = db.cursor()
    c.execute("select content, time from posts order by time desc")
    posts = c.fetchall()
    db.close()
    return posts

def add_post(content):
    """Add a post to the 'database' with the current timestamp."""
    db = psycopg2.connect(database=DBNAME)
    c = db.cursor()
    c.execute("insert into posts values ('%s')" % content) # Almost but not quite.
    db.commit()
    db.close()

A SQL injection attack is whesome of our posts text is being interperited as a database command and used in a way that was non intended. What would happen if we ender the following text into our database? 

`'); delete from posts; --`

The add_post function would read it as c.execute("insert into posts values'); delete from posts; --). Where it would only catch the error after it has already finished the deleted from posts command. We now need to fix this so make sure it doesnt happen again! Lucky for us if you do a %s followed but a tuple with the contents followed by a comma
